# Freshwater eutrophication impact factors 

Calculate impact factors for different types of land stress. IMPORTANT: If all cells are run you might run out of memory. To avoid this only run the ones you want to calculate e.g. the ones tagged with # WATER for calculating impact factors for P emissions to soil. # COMMON cells should always be run.

Exiobase categories:
- P - agriculture - water
- P - agriculture - soil

LC-Impact stressors
- Freshwater eutrophication

In [1]:
# import required libraries
import pymrio
import numpy as np
import pandas as pd
import pycountry as pyc
import json

# load arguments from json file
with open("../arguments.json", "r") as f:
    arguments = json.load(f)

In [3]:
# exiobase 2011 is used for calculating share of stressor for each region-product pair
exio3_11 = pymrio.parse_exiobase3(path=arguments["exio_11_path"])
# exiobase 2019 is used for impact factors
exio3_19 = pymrio.parse_exiobase3(path=arguments["exio_19_path"])

## Calculate DRorigin
DRorigin is a matrix which describes the amount of the driver of biodiversity loss (DR) that occurs in impact region i sector k and is driven by consumption in region j sector k.

To calculate the matrix we need to
1. Aggregate relevant drivers from exiobase together
2. Diagonalize the aggregated driver and re-calculate the IO system

In [ ]:
# no need for aggregation

In [4]:
# COMMON
# diagonalization

# make sure that L matrix is calculated 
if exio3_11.L is None:
    # try loading the L matrix from pickles/exio3_11_L.pickle
    try:
        exio3_11.L = pd.read_pickle("pickles/exio3_11_L.pickle")
        print("L matrix loaded from pickle")
    except FileNotFoundError:
        print("L matrix not found, calculating it from scratch.")
        print("Calculating A")
        exio3_11.A = pymrio.calc_A(exio3_11.Z, exio3_11.x)
        print("Calculating L")
        exio3_11.L = pymrio.calc_L(exio3_11.A)
        # save the L matrix to a pickle file
        pd.to_pickle(exio3_11.L, "pickles/exio3_11_L.pickle")
else:
    print("L already loaded")

Y_agg = exio3_11.Y.groupby(level="region", axis=1, sort=False).sum()

L matrix loaded from pickle


/var/folders/bg/8zzl9cj57fnd569zy539k5h40000gn/T/ipykernel_92308/1974571270.py:21: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  Y_agg = exio3_11.Y.groupby(level="region", axis=1, sort=False).sum()


In [49]:
# WATER
diag_water = exio3_11.satellite.diag_stressor(("P - agriculture - water"))

# calculate S (direct emission multipliers)
if diag_water.S is None:
    print("Calculating S for P emissions to water")
    diag_water.S = pymrio.calc_S(diag_water.F, exio3_11.x)

diag_water.D_cba, _, _, _ = pymrio.calc_accounts(diag_water.S, exio3_11.L, Y_agg)

Calculating S for P emissions to water


In [6]:
# SOIL
diag_soil = exio3_11.satellite.diag_stressor(("P - agriculture - soil"))

# calculate S (direct emission multipliers)
if diag_soil.S is None:
    print("Calculating S for P emissions to soil")
    diag_soil.S = pymrio.calc_S(diag_soil.F, exio3_11.x)

diag_soil.D_cba, _, _, _ = pymrio.calc_accounts(diag_soil.S, exio3_11.L, Y_agg)

Calculating S for P emissions to soil


## Calculate DR share
DR share is a new matrix that represents the share of the driver in the impact region i from the total amount of driver that is driven by consumption in region j sector k.

To calculate the matrix each column of DR origin is shared by the sum of that column.

In [50]:
# WATER
# calculate dr_s - share of the driver of biodiversity loss in impact region i from the total amount of the driver that is driven by consumption in consumption region j, product sector k
columns = {}
for series_name, series in diag_water.D_cba.items():
    series_sum = series.sum()
    columns[series_name] = series / series_sum

dr_s_w = pd.DataFrame(columns)
dr_s_w

AT  \
                                                            Paddy rice   
region sector                                                            
AT     Paddy rice                                         0.000000e+00   
       Wheat                                              9.732484e-08   
       Cereal grains nec                                  4.690399e-07   
       Vegetables, fruit, nuts                            7.602540e-09   
       Oil seeds                                          2.616801e-08   
...                                                                ...   
WM     Membership organisation services n.e.c. (91)       7.440984e-09   
       Recreational, cultural and sporting services (92)  3.407112e-07   
       Other services (93)                                3.350879e-08   
       Private households with employed persons (95)      1.376866e-09   
       Extra-territorial organizations and bodies         0.000000e+00   

                                                                        \
                                                                 Wheat   
region sector                                                            
AT     Paddy rice                                         0.000000e+00   
       Wheat                                              6.371025e-01   
       Cereal grains nec                                  3.556667e-05   
       Vegetables, fruit, nuts                            2.246901e-06   
       Oil seeds                                          4.508837e-06   
...                                                                ...   
WM     Membership organisation services n.e.c. (91)       6.540363e-08   
       Recreational, cultural and sporting services (92)  3.569580e-06   
       Other services (93)                                2.387909e-07   
       Private households with employed persons (95)      1.487512e-08   
       Extra-territorial organizations and bodies         0.000000e+00   

                                                                            \
                                                         Cereal grains nec   
region sector                                                                
AT     Paddy rice                                             0.000000e+00   
       Wheat                                                  1.035024e-05   
       Cereal grains nec                                      7.307470e-01   
       Vegetables, fruit, nuts                                1.558708e-06   
       Oil seeds                                              2.643736e-06   
...                                                                    ...   
WM     Membership organisation services n.e.c. (91)           2.556949e-08   
       Recreational, cultural and sporting services (92)      1.589157e-06   
       Other services (93)                                    9.595777e-08   
       Private households with employed persons (95)          6.241534e-09   
       Extra-territorial organizations and bodies             0.000000e+00   

                                                                                  \
                                                         Vegetables, fruit, nuts   
region sector                                                                      
AT     Paddy rice                                                   0.000000e+00   
       Wheat                                                        4.254184e-05   
       Cereal grains nec                                            1.147146e-04   
       Vegetables, fruit, nuts                                      3.324448e-01   
       Oil seeds                                                    8.766861e-06   
...                                                                          ...   
WM     Membership organisation services n.e.c. (91)                 5.459750e-07   
       Recreational, cultural and sporting services (92)     

In [8]:
# SOIL
# calculate dr_s - share of the driver of biodiversity loss in impact region i from the total amount of the driver that is driven by consumption in consumption region j, product sector k
columns = {}
for series_name, series in diag_soil.D_cba.items():
    series_sum = series.sum()
    columns[series_name] = series / series_sum

dr_s_s = pd.DataFrame(columns)
dr_s_s

AT  \
                                                            Paddy rice   
region sector                                                            
AT     Paddy rice                                         0.000000e+00   
       Wheat                                              9.732484e-08   
       Cereal grains nec                                  4.690399e-07   
       Vegetables, fruit, nuts                            7.602540e-09   
       Oil seeds                                          2.616801e-08   
...                                                                ...   
WM     Membership organisation services n.e.c. (91)       7.440984e-09   
       Recreational, cultural and sporting services (92)  3.407112e-07   
       Other services (93)                                3.350879e-08   
       Private households with employed persons (95)      1.376866e-09   
       Extra-territorial organizations and bodies         0.000000e+00   

                                                                        \
                                                                 Wheat   
region sector                                                            
AT     Paddy rice                                         0.000000e+00   
       Wheat                                              6.371025e-01   
       Cereal grains nec                                  3.556667e-05   
       Vegetables, fruit, nuts                            2.246901e-06   
       Oil seeds                                          4.508837e-06   
...                                                                ...   
WM     Membership organisation services n.e.c. (91)       6.540363e-08   
       Recreational, cultural and sporting services (92)  3.569580e-06   
       Other services (93)                                2.387909e-07   
       Private households with employed persons (95)      1.487512e-08   
       Extra-territorial organizations and bodies         0.000000e+00   

                                                                            \
                                                         Cereal grains nec   
region sector                                                                
AT     Paddy rice                                             0.000000e+00   
       Wheat                                                  1.035024e-05   
       Cereal grains nec                                      7.307470e-01   
       Vegetables, fruit, nuts                                1.558708e-06   
       Oil seeds                                              2.643736e-06   
...                                                                    ...   
WM     Membership organisation services n.e.c. (91)           2.556949e-08   
       Recreational, cultural and sporting services (92)      1.589157e-06   
       Other services (93)                                    9.595777e-08   
       Private households with employed persons (95)          6.241534e-09   
       Extra-territorial organizations and bodies             0.000000e+00   

                                                                                  \
                                                         Vegetables, fruit, nuts   
region sector                                                                      
AT     Paddy rice                                                   0.000000e+00   
       Wheat                                                        4.254184e-05   
       Cereal grains nec                                            1.147146e-04   
       Vegetables, fruit, nuts                                      3.324448e-01   
       Oil seeds                                                    8.766861e-06   
...                                                                          ...   
WM     Membership organisation services n.e.c. (91)                 5.459750e-07   
       Recreational, cultural and sporting services (92)     

## Calculate DR unit
DR unit is a region harmonized version of DR share.

To calculate DR unit we need to:
1. Identify regions that are missing from exiobase (rest of the world regions) but are present in lc-impact.
2. Assume that the impacts are divided evenly across the rest of the world category
3. By that assumption we can add the missing countries to DR share as the impact of the row region where country belongs to divided by the number of countries in that region

In [51]:
# COMMON
# harmonize regions 

exio_regions = exio3_11.get_regions()
row_regions = {
    "WA": "Asia and pacific",
    "WE": "Europe",
    "WF": "Africa",
    "WM": "Middle east",
    "WL": "America"
}

# Function to convert country name to ISO Alpha-2 code
def get_country_code(name):
    # custom mappings for countries that pycountry does not recognize
    # these should cover all the countries in the LCI data if country has alpha-2 code
    # these mappings were extracted manually
    extra_mappings = {
        "Turkey": "TR",
        "Russia": "RU",
        "Bahamas, The": "BS",
        "Bonaire": "BQ",
        "Byelarus": "BY",
        "Brunei": "BN",
        "Cape Verde": "CV",
        "Cocos Islands": "CC",
        "Congo DRC": "CD",
        "China, Hong Kong Special Administrative Region": "HK",
        "Curacao": "CW",
        "Democratic Republic of the Congo": "CD",
        "Falkland Islands": "FK",
        "Falkland Islands (Islas Malvinas)": "FK",
        "Gambia, The": "GM",
        "Gaza Strip": "PS",
        "Heard Island & McDonald Islands": "HM",
        "Ivory Coast": "CI",
        "Macedonia": "MK",
        "The Former Yugoslav Republic of Macedonia": "MK",
        "Macau": "MO",
        "Man, Isle of": "IM",
        "Micronesia": "FM",
        "Myanmar (Burma)": "MM",
        "Netherlands Antilles": "AN",
        "Palestinian Territory": "PS",
        "Pacific Islands (Palau)": "PW",
        "Pitcairn Islands": "PN",
        "Reunion": "RE",
        "Saba": "BQ",
        "Saint Eustatius": "BQ",
        "Saint Helena": "SH",
        "Saint Martin": "MF",
        "Sint Maarten": "SX",
        "South Georgia and the South Sandwich Is": "GS",
        "South Georgia": "GS",
        "St. Helena": "SH",
        "Saint Barthelemy": "BL",
        "Saint Kitts and Nevis": "KN",
        "St. Kitts and Nevis": "KN",
        "St. Lucia": "LC",
        "St. Pierre and Miquelon": "PM",
        "Sao Tomo and Principe": "ST",
        "St. Vincent and the Grenadines": "VC",
        "Svalbard": "SJ",
        "Jan Mayen": "SJ",
        "Swaziland": "SZ",
        "US Virgin Islands": "VI",
        "Virgin Islands": "VG",
        "Western Samoa": "WS",
        "West Bank": "PS",
    }
    try:
        return pyc.countries.lookup(name).alpha_2
    except LookupError:
        try:
            return extra_mappings[name]
        except LookupError:
            print("Alpha-2 country code does not exist for ", name)
            return None

In [52]:
# COMMON
# load and prepare lc-impact data
lci = pd.read_excel(arguments["lc_impact_path"] + "/8-freshwater eutrophication/CF_FWEutrophication.xlsx", sheet_name="Country CFs")
lci.dropna(inplace=True)

# TODO: CF from erosion is not used, is this ok?
lci.rename(columns={lci.columns[1]: "CF for P emissions to water", lci.columns[2]: "CF for P emissions to soil"}, inplace=True)

# Add country codes to the LCI data and drop rows without country codes
lci["Country_Code"] = lci["Country"].apply(get_country_code)
lci.dropna(subset=["Country_Code"], inplace=True)
lci

Alpha-2 country code does not exist for  Zaire


,Country,CF for P emissions to water,CF for P emissions to soil,CF from erosion [PDFyr/m2yr],Country_Code
0,Afghanistan,7.656778e-12,1.063088e-12,6.266043e-12,AF
1,Albania,5.200098e-13,7.508058e-14,1.377819e-12,AL
2,Algeria,6.360048e-15,6.435278e-16,6.584834e-16,DZ
3,Andorra,3.370127e-13,3.111552e-14,2.155679e-13,AD
4,Angola,4.727749e-12,9.038396e-13,1.956920e-12,AO
...,...,...,...,...,...
161,West Bank,1.250468e-12,1.623124e-13,1.903067e-13,PS
162,Western Sahara,5.498873e-13,4.261535e-14,0.000000e+00,EH
163,Yemen,3.315156e-15,2.709144e-16,1.113015e-17,YE
165,Zambia,4.077208e-12,4.462874e-13,2.512662e-12,ZM


In [ ]:
# COMMON
# add regional averages for regions that are not in LCI data

def get_missing_from_lci(exio_regions, lci):
    """
    Get the regions that are in exiobase but not in lci data.
    """
    missing = []
    for region in exio_regions:
        if region not in lci["Country_Code"].tolist():
            missing.append(region)
    return missing

def augment_freshwater(lci_freshwater):
    # malta is missing from lc-impact, add malta as new row with country code MT and EU averages
    p_w_eu = 2.19740701963604E-13
    p_s_eu = 2.27816881528299E-14
    malta_row = pd.DataFrame({
        "Country": ["Malta"],
        "CF for P emissions to water": [p_w_eu],
        "CF for P emissions to soil": [p_s_eu],
        "Country_Code": ["MT"],
    })
    lci_freshwater = pd.concat([lci_freshwater, malta_row], ignore_index=True)
    # cyprus is missing from lc-impact, add cyprus as new row with country code CY and EU averages
    cyprus_row = pd.DataFrame({
        "Country": ["Cyprus"],
        "CF for P emissions to water": [p_w_eu],
        "CF for P emissions to soil": [p_s_eu],
        "Country_Code": ["CY"],
    })
    lci_freshwater = pd.concat([lci_freshwater, cyprus_row], ignore_index=True)
    return lci_freshwater

exio_regions_without_row = [region for region in exio_regions if region not in row_regions.keys()]
if len(get_missing_from_lci(exio_regions_without_row, lci)) > 0:
    print("Missing from LCI:", get_missing_from_lci(exio_regions_without_row, lci))
    lci = augment_freshwater(lci)
    assert len(get_missing_from_lci(exio_regions_without_row, lci)) == 0, "There are still missing regions after augmentation"

Missing from LCI land stress: ['CY', 'MT']


In [55]:
# COMMON
# harmonize regions in LCI data

def get_row_regions(lci_country_codes, exio_country_codes):
    """
    Get the country codes from lci countries that don't exist in exiobase i.e. rest of the world countries.
    """
    row_regions = []
    for country in lci_country_codes:
        if country not in exio_country_codes:
            row_regions.append(country)
    
    # find duplicates in the list
    duplicates = []
    unique_regions = []
    seen_once = set()
    for item in row_regions:
        if item not in seen_once:
            unique_regions.append(item)
            seen_once.add(item)
        else:
            duplicates.append(item)
    if duplicates:
        print("Duplicates found in row regions:", duplicates)
    return unique_regions

row_countries = get_row_regions(lci["Country_Code"].tolist(), exio_regions)
print("Row regions:", row_countries)

# Load region mappings from arguments
row_eu_countries = arguments["row_region_mappings"]["row_eu"]
row_asia_pacific_countries = arguments["row_region_mappings"]["row_asia_pacific"]
row_african_countries = arguments["row_region_mappings"]["row_africa"]
row_american_countries = arguments["row_region_mappings"]["row_america"]
row_middle_eastern_countries = arguments["row_region_mappings"]["row_middle_east"]

Duplicates found in row regions: ['PS']
Row regions: ['AF', 'AL', 'DZ', 'AD', 'AO', 'AR', 'AM', 'AZ', 'BH', 'BD', 'BZ', 'BJ', 'BT', 'BO', 'BA', 'BW', 'BN', 'BF', 'BI', 'BY', 'KH', 'CM', 'CF', 'TD', 'CL', 'CO', 'CG', 'CR', 'CU', 'DJ', 'DO', 'EC', 'EG', 'SV', 'GQ', 'ER', 'ET', 'GF', 'GA', 'GM', 'PS', 'GE', 'GH', 'GT', 'GN', 'GW', 'GY', 'HT', 'HN', 'IS', 'IR', 'IQ', 'IL', 'CI', 'JO', 'KZ', 'KE', 'KW', 'KG', 'LA', 'LB', 'LS', 'LR', 'LY', 'LI', 'MK', 'MG', 'MW', 'MY', 'ML', 'MR', 'MD', 'MN', 'ME', 'MA', 'MZ', 'MM', 'NA', 'NP', 'NZ', 'NI', 'NE', 'NG', 'KP', 'OM', 'PK', 'PA', 'PG', 'PY', 'PE', 'PH', 'QA', 'RW', 'SM', 'SA', 'SN', 'RS', 'SL', 'SG', 'SO', 'LK', 'SD', 'SR', 'SZ', 'SY', 'TJ', 'TZ', 'TH', 'TG', 'TT', 'TN', 'TM', 'UG', 'UA', 'AE', 'UY', 'UZ', 'VE', 'VN', 'EH', 'YE', 'ZM', 'ZW']


In [57]:
# WATER
# augment dr_s_w to create dr_u_w

# new regions are calculated by dividing their corresponding row region by the number of countries in the row region
# for example, row region Argentina is sub-matrix WA divided by the number of countries in row region WA
wl_water = dr_s_w.loc["WL"].copy()
wl_water = wl_water / len(row_american_countries)

we_water = dr_s_w.loc["WE"].copy()
we_water = we_water / len(row_eu_countries)

wa_water = dr_s_w.loc["WA"].copy()
wa_water = wa_water / len(row_asia_pacific_countries)

wf_water = dr_s_w.loc["WF"].copy()
wf_water = wf_water / len(row_african_countries)

wm_water = dr_s_w.loc["WM"].copy()
wm_water = wm_water / len(row_middle_eastern_countries)

dr_u_w = dr_s_w.copy()
dr_u_w = dr_u_w.drop(index=row_regions.keys(), level='region')

# build a mapping of country codes to region dataframes
country_to_region_water = {}
for region in row_countries:
    if region in row_eu_countries:
        country_to_region_water[region] = we_water
    elif region in row_asia_pacific_countries:
        country_to_region_water[region] = wa_water
    elif region in row_african_countries:
        country_to_region_water[region] = wf_water
    elif region in row_american_countries:
        country_to_region_water[region] = wl_water
    elif region in row_middle_eastern_countries:
        country_to_region_water[region] = wm_water
    else:
        raise ValueError(f"Unknown region: {region}")

# add all new regions to dr_u_w
all_indices = []
all_data = []
for region in row_countries:
    region_data = country_to_region_water[region].copy()
    idx = pd.MultiIndex.from_product([[region],region_data.index], names=['region', 'sector'])
    all_indices.append(idx)
    all_data.append(region_data)

combined_idx_water = pd.MultiIndex.from_tuples(
    [idx for subidx in all_indices for idx in subidx]
)

combined_data_water = pd.concat(all_data)
combined_data_water.index = combined_idx_water

dr_u_w = pd.concat([dr_u_w, combined_data_water])
# drop row region columns
dr_u_w = dr_u_w.drop(columns=row_regions.keys(), axis=1, level=0)
dr_u_w

AT  \
                                                        Paddy rice   
AT Paddy rice                                         0.000000e+00   
   Wheat                                              9.732484e-08   
   Cereal grains nec                                  4.690399e-07   
   Vegetables, fruit, nuts                            7.602540e-09   
   Oil seeds                                          2.616801e-08   
...                                                            ...   
ZW Membership organisation services n.e.c. (91)       6.000704e-10   
   Recreational, cultural and sporting services (92)  4.286236e-09   
   Other services (93)                                8.591412e-10   
   Private households with employed persons (95)      5.633862e-11   
   Extra-territorial organizations and bodies         0.000000e+00   

                                                                    \
                                                             Wheat   
AT Paddy rice                                         0.000000e+00   
   Wheat                                              6.371025e-01   
   Cereal grains nec                                  3.556667e-05   
   Vegetables, fruit, nuts                            2.246901e-06   
   Oil seeds                                          4.508837e-06   
...                                                            ...   
ZW Membership organisation services n.e.c. (91)       1.489729e-08   
   Recreational, cultural and sporting services (92)  1.476885e-07   
   Other services (93)                                1.672662e-08   
   Private households with employed persons (95)      1.373946e-09   
   Extra-territorial organizations and bodies         0.000000e+00   

                                                                        \
                                                     Cereal grains nec   
AT Paddy rice                                             0.000000e+00   
   Wheat                                                  1.035024e-05   
   Cereal grains nec                                      7.307470e-01   
   Vegetables, fruit, nuts                                1.558708e-06   
   Oil seeds                                              2.643736e-06   
...                                                                ...   
ZW Membership organisation services n.e.c. (91)           5.976464e-09   
   Recreational, cultural and sporting services (92)      6.792702e-08   
   Other services (93)                                    6.744450e-09   
   Private households with employed persons (95)          4.573571e-10   
   Extra-territorial organizations and bodies             0.000000e+00   

                                                                              \
                                                     Vegetables, fruit, nuts   
AT Paddy rice                                                   0.000000e+00   
   Wheat                                                        4.254184e-05   
   Cereal grains nec                                            1.147146e-04   
   Vegetables, fruit, nuts                                      3.324448e-01   
   Oil seeds                                                    8.766861e-06   
...                                                                      ...   
ZW Membership organisation services n.e.c. (91)                 7.416828e-08   
   Recreational, cultural and sporting services (92)            1.863727e-07   
   Other services (93)                                          2.433587e-08   
   Private households with employed persons (95)                1.595285e-09   
   Extra-territorial organizations and bodies                   0.000000e+00   

                                                                    \
                                                         Oil seeds   
AT Paddy rice                                         0.000000e+00   
   Wheat                    

In [35]:
# SOIL
# augment dr_s_s to create dr_u_s

# new regions are calculated by dividing their corresponding row region by the number of countries in the row region
wl_soil = dr_s_s.loc["WL"].copy()
wl_soil = wl_soil / len(row_american_countries)

we_soil = dr_s_s.loc["WE"].copy()
we_soil = we_soil / len(row_eu_countries)

wa_soil = dr_s_s.loc["WA"].copy()
wa_soil = wa_soil / len(row_asia_pacific_countries)

wf_soil = dr_s_s.loc["WF"].copy()
wf_soil = wf_soil / len(row_african_countries)

wm_soil = dr_s_s.loc["WM"].copy()
wm_soil = wm_soil / len(row_middle_eastern_countries)

dr_u_s = dr_s_s.copy()
dr_u_s = dr_u_s.drop(index=row_regions.keys(), level='region')

# build a mapping of country codes to region dataframes
country_to_region_soil = {}
for region in row_countries:
    if region in row_eu_countries:
        country_to_region_soil[region] = we_soil
    elif region in row_asia_pacific_countries:
        country_to_region_soil[region] = wa_soil
    elif region in row_african_countries:
        country_to_region_soil[region] = wf_soil
    elif region in row_american_countries:
        country_to_region_soil[region] = wl_soil
    elif region in row_middle_eastern_countries:
        country_to_region_soil[region] = wm_soil
    else:
        raise ValueError(f"Unknown region: {region}")

# add all new regions to dr_u_s
all_indices = []
all_data = []
for region in row_countries:
    region_data = country_to_region_soil[region].copy()
    idx = pd.MultiIndex.from_product([[region], region_data.index], names=['region', 'sector'])
    all_indices.append(idx)
    all_data.append(region_data)

combined_idx = pd.MultiIndex.from_tuples(
    [idx for subidx in all_indices for idx in subidx]
)

combined_data = pd.concat(all_data)
combined_data.index = combined_idx

dr_u_s = pd.concat([dr_u_s, combined_data])
# drop row region columns
dr_u_s = dr_u_s.drop(columns=row_regions.keys(), axis=1, level=0)
dr_u_s


AT  \
                                                        Paddy rice   
AT Paddy rice                                         0.000000e+00   
   Wheat                                              9.732484e-08   
   Cereal grains nec                                  4.690399e-07   
   Vegetables, fruit, nuts                            7.602540e-09   
   Oil seeds                                          2.616801e-08   
...                                                            ...   
ZW Membership organisation services n.e.c. (91)       6.000704e-10   
   Recreational, cultural and sporting services (92)  4.286236e-09   
   Other services (93)                                8.591412e-10   
   Private households with employed persons (95)      5.633862e-11   
   Extra-territorial organizations and bodies         0.000000e+00   

                                                                    \
                                                             Wheat   
AT Paddy rice                                         0.000000e+00   
   Wheat                                              6.371025e-01   
   Cereal grains nec                                  3.556667e-05   
   Vegetables, fruit, nuts                            2.246901e-06   
   Oil seeds                                          4.508837e-06   
...                                                            ...   
ZW Membership organisation services n.e.c. (91)       1.489729e-08   
   Recreational, cultural and sporting services (92)  1.476885e-07   
   Other services (93)                                1.672662e-08   
   Private households with employed persons (95)      1.373946e-09   
   Extra-territorial organizations and bodies         0.000000e+00   

                                                                        \
                                                     Cereal grains nec   
AT Paddy rice                                             0.000000e+00   
   Wheat                                                  1.035024e-05   
   Cereal grains nec                                      7.307470e-01   
   Vegetables, fruit, nuts                                1.558708e-06   
   Oil seeds                                              2.643736e-06   
...                                                                ...   
ZW Membership organisation services n.e.c. (91)           5.976464e-09   
   Recreational, cultural and sporting services (92)      6.792702e-08   
   Other services (93)                                    6.744450e-09   
   Private households with employed persons (95)          4.573571e-10   
   Extra-territorial organizations and bodies             0.000000e+00   

                                                                              \
                                                     Vegetables, fruit, nuts   
AT Paddy rice                                                   0.000000e+00   
   Wheat                                                        4.254184e-05   
   Cereal grains nec                                            1.147146e-04   
   Vegetables, fruit, nuts                                      3.324448e-01   
   Oil seeds                                                    8.766861e-06   
...                                                                      ...   
ZW Membership organisation services n.e.c. (91)                 7.416828e-08   
   Recreational, cultural and sporting services (92)            1.863727e-07   
   Other services (93)                                          2.433587e-08   
   Private households with employed persons (95)                1.595285e-09   
   Extra-territorial organizations and bodies                   0.000000e+00   

                                                                    \
                                                         Oil seeds   
AT Paddy rice                                         0.000000e+00   
   Wheat                    

## Calculate DR factor
Calculate the impact factors of the driver for each impact region i driven by consumption in region j product sector k.

These impact factors tell how

To calculate DR factor:
1. Calculate the monetary impact factor impact/€ from 2019 exiobase data
2. Multiply each column of DR unit with the impact factors of consumption region j in product sector k. The resulting matrix represents the distribution of impacts of 1€ consumption in each consumption region.

In [58]:
# WATER

# use 2019 impact factors for calculating dr_f
# calculate dr_f - share of the driver of biodiversity loss in impact region i from the total amount of the driver that is driven by consumption in consumption region j, product sector k
dr_f_w = dr_u_w.copy()
satellite_cleaned = exio3_19.satellite.M.drop(columns=row_regions.keys(), axis=1, level=0)
total = satellite_cleaned.loc["P - agriculture - water"]
scalars = total.to_numpy() # multipliers for each column
print(len(scalars))
print(dr_u_w.shape)

# multiply each column of dr_u_w by the respective column value
dr_f_w = dr_f_w * scalars
dr_f_w

8800
(33400, 8800)


AT                \
                                                     Paddy rice         Wheat   
AT Paddy rice                                               0.0  0.000000e+00   
   Wheat                                                    0.0  6.218397e+01   
   Cereal grains nec                                        0.0  3.471461e-03   
   Vegetables, fruit, nuts                                  0.0  2.193073e-04   
   Oil seeds                                                0.0  4.400820e-04   
...                                                         ...           ...   
ZW Membership organisation services n.e.c. (91)             0.0  1.454040e-06   
   Recreational, cultural and sporting services (92)        0.0  1.441504e-05   
   Other services (93)                                      0.0  1.632590e-06   
   Private households with employed persons (95)            0.0  1.341031e-07   
   Extra-territorial organizations and bodies               0.0  0.000000e+00   

                                                                        \
                                                     Cereal grains nec   
AT Paddy rice                                             0.000000e+00   
   Wheat                                                  1.038944e-03   
   Cereal grains nec                                      7.335142e+01   
   Vegetables, fruit, nuts                                1.564610e-04   
   Oil seeds                                              2.653748e-04   
...                                                                ...   
ZW Membership organisation services n.e.c. (91)           5.999096e-07   
   Recreational, cultural and sporting services (92)      6.818425e-06   
   Other services (93)                                    6.769990e-07   
   Private households with employed persons (95)          4.590890e-08   
   Extra-territorial organizations and bodies             0.000000e+00   

                                                                              \
                                                     Vegetables, fruit, nuts   
AT Paddy rice                                                   0.000000e+00   
   Wheat                                                        8.147880e-04   
   Cereal grains nec                                            2.197086e-03   
   Vegetables, fruit, nuts                                      6.367191e+00   
   Oil seeds                                                    1.679084e-04   
...                                                                      ...   
ZW Membership organisation services n.e.c. (91)                 1.420517e-06   
   Recreational, cultural and sporting services (92)            3.569527e-06   
   Other services (93)                                          4.660958e-07   
   Private households with employed persons (95)                3.055391e-08   
   Extra-territorial organizations and bodies                   0.000000e+00   

                                                                    \
                                                         Oil seeds   
AT Paddy rice                                         0.000000e+00   
   Wheat                                              1.153229e-04   
   Cereal grains nec                                  2.208263e-04   
   Vegetables, fruit, nuts                            1.599124e-05   
   Oil seeds                                          5.198346e-01   
...                                                            ...   
ZW Membership organisation services n.e.c. (91)       3.353884e-07   
   Recreational, cultural and sporting services (92)  2.394463e-06   
   Other services (93)                                3.547217e-07   
   Private households with employed persons (95)      3.886449e-08   
   Extra-territorial organizations and bodies         0.000000e+00   

                                                                             \
             

In [37]:
# SOIL

# use 2019 impact factors for calculating dr_f
dr_f_s = dr_u_s.copy()
satellite_cleaned = exio3_19.satellite.M.drop(columns=row_regions.keys(), axis=1, level=0)
total = satellite_cleaned.loc["P - agriculture - soil"]
scalars = total.to_numpy() # multipliers for each column
print(len(scalars))
print(dr_u_s.shape)

# multiply each column of dr_u_s by the respective column value
dr_f_s = dr_f_s * scalars
dr_f_s


8800
(33400, 8800)


AT               \
                                                     Paddy rice        Wheat   
AT Paddy rice                                               0.0     0.000000   
   Wheat                                                    0.0  2082.090747   
   Cereal grains nec                                        0.0     0.116234   
   Vegetables, fruit, nuts                                  0.0     0.007343   
   Oil seeds                                                0.0     0.014735   
...                                                         ...          ...   
ZW Membership organisation services n.e.c. (91)             0.0     0.000049   
   Recreational, cultural and sporting services (92)        0.0     0.000483   
   Other services (93)                                      0.0     0.000055   
   Private households with employed persons (95)            0.0     0.000004   
   Extra-territorial organizations and bodies               0.0     0.000000   

                                                                        \
                                                     Cereal grains nec   
AT Paddy rice                                                 0.000000   
   Wheat                                                      0.034787   
   Cereal grains nec                                       2456.008052   
   Vegetables, fruit, nuts                                    0.005239   
   Oil seeds                                                  0.008885   
...                                                                ...   
ZW Membership organisation services n.e.c. (91)               0.000020   
   Recreational, cultural and sporting services (92)          0.000228   
   Other services (93)                                        0.000023   
   Private households with employed persons (95)              0.000002   
   Extra-territorial organizations and bodies                 0.000000   

                                                                              \
                                                     Vegetables, fruit, nuts   
AT Paddy rice                                                       0.000000   
   Wheat                                                            0.027281   
   Cereal grains nec                                                0.073564   
   Vegetables, fruit, nuts                                        213.191131   
   Oil seeds                                                        0.005622   
...                                                                      ...   
ZW Membership organisation services n.e.c. (91)                     0.000048   
   Recreational, cultural and sporting services (92)                0.000120   
   Other services (93)                                              0.000016   
   Private households with employed persons (95)                    0.000001   
   Extra-territorial organizations and bodies                       0.000000   

                                                                 \
                                                      Oil seeds   
AT Paddy rice                                          0.000000   
   Wheat                                               0.003861   
   Cereal grains nec                                   0.007394   
   Vegetables, fruit, nuts                             0.000535   
   Oil seeds                                          17.405496   
...                                                         ...   
ZW Membership organisation services n.e.c. (91)        0.000011   
   Recreational, cultural and sporting services (92)   0.000080   
   Other services (93)                                 0.000012   
   Private households with employed persons (95)       0.000001   
   Extra-territorial organizations and bodies          0.000000   

                                                                             \
                                                     Sugar cane, 

## Calculate BDe factors (PDF/€ for each consumption region and product sector)

1. Multiply each cell of dr_u with the CF (characterisation factor from lc-impact) of the impact region to get PDF values for every entry
2. Sum up columns to get the total PDF/€ for the consumption region j product sector k

In [60]:
# WATER
lci_copy_water = lci.copy()
lci_copy_water.set_index("Country_Code", inplace=True)

# Ensure lci index is unique before reindexing
lci_copy_water = lci_copy_water[~lci_copy_water.index.duplicated(keep='first')]

# sort rows on lci in same order as dr_f_w
lci_reindexed_water = lci_copy_water.reindex(dr_f_w.index.get_level_values(0).unique())

cf_water = lci_reindexed_water["CF for P emissions to water"].to_numpy()
cf_water = np.repeat(cf_water, 200)
cf_water = np.tile(cf_water, (dr_f_w.shape[1], 1)).T

pdf_water = dr_f_w * cf_water
pdf_total_water = pdf_water.sum()

In [ ]:
# SOIL
lci_copy_soil = lci.copy()
lci_copy_soil.set_index("Country_Code", inplace=True)

# Ensure lci index is unique before reindexing
lci_copy_soil = lci_copy_soil[~lci_copy_soil.index.duplicated(keep='first')]

# sort rows on lci in same order as dr_f_s
lci_reindexed_soil = lci_copy_soil.reindex(dr_f_s.index.get_level_values(0).unique())

cf_soil = lci_reindexed_soil["CF for P emissions to soil"].to_numpy()
cf_soil = np.repeat(cf_soil, 200)
cf_soil = np.tile(cf_soil, (dr_f_s.shape[1], 1)).T

pdf_soil = dr_f_s * cf_soil
pdf_total_soil = pdf_soil.sum()

In [ ]:
# WATER
pd.DataFrame(pdf_total_water).to_csv("csv/pdf-eutrophication-freshwater-water.csv", index=True)

In [ ]:
# SOIL
pd.DataFrame(pdf_total_soil).to_csv("csv/pdf-eutrophication-freshwater-soil.csv", index=True)